In [659]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import NoSuchElementException
import chromedriver_binary
import os
import pandas as pd
import numpy as np
import ipynb.fs.full.Project2ScraperUtil as util
import importlib as imlib
import pprint as pprint

In [660]:
imlib.reload(util)

<module 'ipynb.fs.full.Project2ScraperUtil' (/Users/nmangla/Desktop/Metis_Course/SGP19_DS0/curriculum/project-02/project-02-introduction/Project2ScraperUtil.ipynb)>

In [661]:
def getBookInfo(driver):
    dictionary = {}
    
    # Get all information from book page
    dictionary = util.getBookInfo(driver, dictionary)
    if dictionary == None:
        return None
        
    return dictionary

In [662]:
def getBookGenresInfo(driver, dictionary):
    # Get all genres of book
    try:
        genresButton = driver.find_element_by_xpath("//div[@class='stacked']/div[1]/div[1]/h2[1]/a[1]")
        genresButton.click()
        waitForPage = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'leftContainer')))        
        util.wait()
        dictionary = util.getBookGenres(driver, dictionary)
        
        driver.back()
        waitForPage = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID,'bookTitle')))        
        util.wait()
    except NoSuchElementException:
        dictionary[util.getGenres()] = np.nan
        dictionary[util.getGenresCount()] = 0
        pass
    
    return dictionary

In [481]:
# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

In [482]:
# Opening home page to login
driver = webdriver.Chrome()
driver.get('https://www.goodreads.com/')
util.wait()

In [483]:
%load_ext dotenv
%dotenv

# Loading username and password from .env file
EMAIL = os.environ.get('USERNAME')
PASSWORD = os.environ.get('PASSWORD')

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [484]:
# Filling username
usernameForm = driver.find_element_by_id('userSignInFormEmail')
usernameForm.send_keys(EMAIL)
util.wait()

In [485]:
# Filling password
passwordForm=driver.find_element_by_id('user_password')
passwordForm.send_keys(PASSWORD)
util.wait()

In [486]:
# Hitting Enter to login
passwordForm.send_keys(Keys.RETURN)
util.wait()

In [487]:
# Going to list: Best Fantasy Books of the 21st Century
driver.get('https://www.goodreads.com/list/show/88.Best_Fantasy_Books_of_the_21st_Century')
util.wait()

In [706]:
page = 23
startingPoint = 58

# Iterate through books and store information in JSON file
while startingPoint <= 100:
    
    # Find book
    xpath = "//div[@id='all_votes']/table[1]/tbody[1]/tr[" + str(startingPoint) + "]"
    b = driver.find_element_by_xpath(xpath)
    bookNumber = int(b.find_element_by_xpath(".//td[1]").text)
    
    book = b.find_element_by_xpath(".//td[3]/a")
    print(book.text, ' Page:', page, ' Book No:', bookNumber, ' Start No:', startingPoint)
    
    # Click on book, wait for it to load. Then wait some more
    book.click()    
    waitForPage = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID,'bookTitle')))        
    util.wait()  
    
    dictionary = getBookInfo(driver)
    if dictionary == None:
        print("Something went wrong. Skipping this one")
    else:
        dictionary = getBookGenresInfo(driver, dictionary)
        print("Appending to file")
        result = util.appendToFile(dictionary)
        if result == False:
            print("Something messed up!")
            break
        else:
            print("Success!")
    
    driver.back()
    waitForPage = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID,'all_votes')))        
    util.wait()
    startingPoint += 1

Crimson Death (Anita Blake, Vampire Hunter #25)  Page: 23  Book No: 2257  Start No: 58
Appending to file
Success!
Spellbound Falls (Spellbound Falls #1)  Page: 23  Book No: 2259  Start No: 59
Appending to file
Success!
Half-Off Ragnarok (InCryptid, #3)  Page: 23  Book No: 2259  Start No: 60
Appending to file
Success!
Miles Errant (Vorkosigan Omnibus, #4)  Page: 23  Book No: 2261  Start No: 61
Something went wrong. Skipping this one
Everlastin' (Everlastin', #1)  Page: 23  Book No: 2261  Start No: 62
Appending to file
Success!
The Future Falls (Gale Women, #3)  Page: 23  Book No: 2261  Start No: 63
Appending to file
Success!
Dusk Before Dawn (Everlastin' #2)  Page: 23  Book No: 2264  Start No: 64
Appending to file
Success!
Blood Magick (The Cousins O'Dwyer Trilogy, #3)  Page: 23  Book No: 2264  Start No: 65
Appending to file
Success!
The White Dragon (Chronicles of Sirkara, #2)  Page: 23  Book No: 2266  Start No: 66
Appending to file
Success!
Midnight Blue-Light Special (InCryptid, #2) 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='all_votes']/table[1]/tbody[1]/tr[85]"}
  (Session info: chrome=75.0.3770.142)
  (Driver info: chromedriver=2.38.552518 (183d19265345f54ce39cbb94cf81ba5f15905011),platform=Mac OS X 10.12.2 x86_64)


In [707]:
df = util.readFile()

In [708]:
df.tail(1)

,title,series,seriesPosition,seriesNumBooks,author,authorNumFollowers,rating,numRatings,numReviews,numOneStars,numTwoStars,numThreeStars,numFourStars,numFiveStars,percOneStars,percTwoStars,percThreeStars,percFourStars,percFiveStars,numPages,publishDate,isbn,isbn13,asin,publisher,numAwards,awards,genres,genresCount
2137,Visions,Cainsville,2,5,Kelley Armstrong,26082,4.25,10238,909,41,167,1282,4432,4316,0,1,12,43,42,448,2014-08-14 00:00:00,0525953051,9.781e+12,NaN,Dutton,0,NaN,"['to-read', 'urban-fantasy', 'fantasy', 'paran...","[405, 323, 282, 223, 189, 137, 119, 91, 91, 89..."
